<a href="https://colab.research.google.com/github/melataguia/regretion_cluster../blob/main/random_forest_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation des bibliothèques nécessaires

In [ ]:
getwd()
if (!require(randomForest)) {
  install.packages("randomForest")
}

if (!require(dplyr)) {
  install.packages("dplyr")
}
if (!require(stringr)) {
  install.packages("stringr")
}





# Chargement des bibliothèques nécessaires

In [ ]:
library(dplyr)
library(randomForest)
library(ggplot2)
library(stringr)



# Chargement la fonction combine de randomForest


In [5]:
combine <- randomForest::combine

###  Lire les fichiers CSV

In [6]:
movies_df <- read.csv('../movies.csv')
ratings_df <- read.csv('../ratings.csv')
#tags <- read.csv('./tags.csv')


In [7]:
length(ratings_df)

[1] 4

In [8]:
length(movies_df)

[1] 3

### Vérification les données manquantes

In [9]:
missing_values <- sapply(ratings_df, function(x) sum(is.na(x)))
print(missing_values)

   userId   movieId    rating timestamp 
        0         0         0         0 


# Supprimer les lignes avec des valeurs manquantes

In [11]:
ratings_df <- ratings_df[complete.cases(ratings_df), ]


##Fusionner les dataframes sur la colonne "movieid"

In [12]:
merged_df <- merge(ratings_df, movies_df, by="movieId", all.x=TRUE)

## Sélection des colonnes pertinentes pour l'entraînement du modèle

In [13]:
selected_columns <- c("movieId", "userId", "genres", "rating")
train_data <- merged_df[selected_columns]

#Encodage des variables catégorielles (genre)

# Encodage one-hot pour la colonne "genre"

In [14]:
encoded_genre <- model.matrix(~ genres - 1, data = train_data)

# Ajout des colonnes encodées au dataframe

In [15]:
train_data <- cbind(train_data, encoded_genre)

### Division des données en ensembles d'apprentissage et de test

In [16]:
set.seed(42)  # Pour la reproductibilité
train_indices <- sample(1:nrow(train_data), 0.8 * nrow(train_data))
train_set <- train_data[train_indices, ]
test_set <- train_data[-train_indices, ]

#Entraînement du modèle Random Forest

###Séparation de la variable cible (ratings) des caractéristiques

In [17]:
X_train <- train_set[, c("movieId", "userId", colnames(encoded_genre))]
y_train <- train_set$rating

In [ ]:
colnames(X_train)

In [19]:
# Renommer la colonne en retirant les espaces et les caractères spéciaux
colnames(X_train) <- make.names(colnames(X_train))

In [ ]:
colnames(X_train)

In [21]:
unique(X_train$`genres(no genres listed)`)

NULL

## Entraînement du modèle

In [ ]:
rf_model <- randomForest(y_train ~ ., data = X_train, ntree = 20)

#Évaluation du modèle sur l'ensemble de test

###Séparer la variable cible (ratings) des caractéristiques dans l'ensemble de test



In [ ]:
X_test <- test_set[, c("movieid", "userid", colnames(encoded_genre))]
y_test <- test_set$ratings

###Faire des prédictions


In [ ]:
predictions <- predict(rf_model, newdata = X_test)



###Évaluer la performance avec le RMSE


In [ ]:
rmse <- sqrt(mean((predictions - y_test)^2))
print(paste("RMSE sur l'ensemble de test :", rmse))